# Tarea 01 **Polars y gráficas**

#### Informe Estadístico: Análisis de Servicios de Taxi en Enero

A continuación, se presentan los principales hallazgos y visualizaciones del análisis realizado:

 1. Promedio de ingresos brutos de cada tipo de taxi y de todos los taxis

 2. Qué tipo de taxis recorrieron la mayor cantidad de distancia en Enero

 3. Qué tipo de taxi recibe más propinas en promedio

 4. Gráfico de barras reportando el importe del peaje total de cada tipo de Taxi

 5. Un mapa de calor que describa el comportamiento de a qué hora se tomaron todos los taxis durante los días de Enero

 6. Un mapa de calor para cada tipo de taxi que reporte los días de Enero en los que más se uso su servicio

In [1]:
import polars as pl
import matplotlib.pyplot as mpl

***Promedio*** de ingresos brutos de cada tipo de taxi y de todos los taxis

In [2]:
yellow = pl.read_parquet("yellow_tripdata_2025-01.parquet")
green = pl.read_parquet("green_tripdata_2025-01.parquet")

yellow = yellow.select(pl.col("total_amount").mean().alias("Promedio taxi amarillo"))
green = green.select(pl.col("total_amount").mean().alias("Promedio taxi verde"))
total = ((yellow + green)/2)
print(yellow)
print(green)
print(total)

shape: (1, 1)
┌────────────────────────┐
│ Promedio taxi amarillo │
│ ---                    │
│ f64                    │
╞════════════════════════╡
│ 25.611292              │
└────────────────────────┘
shape: (1, 1)
┌─────────────────────┐
│ Promedio taxi verde │
│ ---                 │
│ f64                 │
╞═════════════════════╡
│ 22.634242           │
└─────────────────────┘
shape: (1, 1)
┌────────────────────────┐
│ Promedio taxi amarillo │
│ ---                    │
│ f64                    │
╞════════════════════════╡
│ 24.122767              │
└────────────────────────┘


Qué ***tipo*** de taxis recorrieron la mayor cantidad de distancia en Enero

In [3]:
yellow = pl.read_parquet("yellow_tripdata_2025-01.parquet")
green = pl.read_parquet("green_tripdata_2025-01.parquet")

distance_summary = pl.concat([
    yellow.select(
        pl.col("trip_distance").sum().alias("distance"),
        pl.lit("amarillo").alias("taxi_type")
    ),
    green.select(
        pl.col("trip_distance").sum().alias("distance"),
        pl.lit("verde").alias("taxi_type")
    )
])

# Resultado en un solo DataFrame
print(distance_summary)

shape: (2, 2)
┌──────────┬───────────┐
│ distance ┆ taxi_type │
│ ---      ┆ ---       │
│ f64      ┆ str       │
╞══════════╪═══════════╡
│ 2.0348e7 ┆ amarillo  │
│ 1.0406e6 ┆ verde     │
└──────────┴───────────┘
